<a href="https://colab.research.google.com/github/kaiyungtan/challenge-data-analysis/blob/master/challenge_data_analysis_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge: Data Cleaning

## Imports

In [250]:
# allow jupyter notebook to print all the interactive output 
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [251]:
import pandas as pd
from pandas_profiling import ProfileReport
import missingno as msno

In [252]:
DATASET_URL = "https://raw.githubusercontent.com/kaiyungtan/challenge-data-analysis/master/data/raw/dataset_house_apartment.csv"
POSTAL_CODE_URL = "https://raw.githubusercontent.com/kaiyungtan/challenge-data-analysis/master/data/raw/code-postaux-belge.csv"


df = pd.read_csv(DATASET_URL, sep = ',')
postal_codes = pd.read_csv(POSTAL_CODE_URL, sep = ';')

In [253]:
df.shape
postal_codes.shape

(52077, 20)

(2757, 6)

## Postal Code Aggregation

In [254]:
# Drop empty columns
postal_codes.drop(columns=['coordonnees', 'geom'], inplace=True)

# Rename the columns
postal_codes.rename(columns={"column_1": "locality", "column_2": "city_name", 'column_3': 'lattitude', 'column_4': 'longitude'}, inplace=True)

# Drop the localities' duplicates
postal_codes.drop_duplicates(subset=['locality'], inplace=True)

In [255]:
# Merge solution 1:
df = pd.merge(df, postal_codes, on=['locality'], how='inner')

# Merge solution 2:
# df.merge(postal_codes, on=['locality'], how='inner', copy=True)

# Merge solution 3 (Merge only one column and add it to the dataframe)
# df['city_name'] = df['locality'].map(postal_codes.set_index('locality')['city_name'])

df.head(5)

,locality,type_of_property,subtype_of_property,price,type_of_sale,number_of_rooms,house_area,fully_equipped_kitchen,furnished,open_fire,terrace,terrace_area,garden,garden_area,surface_of_the_land,surface_of_the_plot_of_land,number_of_facades,swimming_pool,state_of_the_building,construction_year,city_name,lattitude,longitude
0,1050,house,house,340000,for sale,6.0,203,1,None,0,1,None,0,None,95.0,None,2,0,to be done up,1901,Ixelles,4.381571,50.822285
1,1050,house,mixed use building,520000,for sale,4.0,200,0,None,0,0,None,0,None,69.0,None,2,0,to renovate,1940,Ixelles,4.381571,50.822285
2,1050,house,house,no price,for sale,6.0,None,1,None,0,0,None,0,None,0.0,None,None,0,None,1910,Ixelles,4.381571,50.822285
3,1050,house,house,599000,for sale,4.0,160,1,None,0,1,None,1,55,100.0,None,2,0,to be done up,1898,Ixelles,4.381571,50.822285
4,1050,house,house,599000,for sale,3.0,160,1,None,0,1,15,1,60,130.0,None,2,0,good,1953,Ixelles,4.381571,50.822285


In [256]:
df.price.dtypes

dtype('O')

In [257]:
df.dtypes

locality                         int64
type_of_property                object
subtype_of_property             object
price                           object
type_of_sale                    object
number_of_rooms                float64
house_area                      object
fully_equipped_kitchen           int64
furnished                       object
open_fire                        int64
terrace                          int64
terrace_area                    object
garden                           int64
garden_area                     object
surface_of_the_land            float64
surface_of_the_plot_of_land     object
number_of_facades               object
swimming_pool                    int64
state_of_the_building           object
construction_year               object
city_name                       object
lattitude                      float64
longitude                      float64
dtype: object

## Drop duplicates row , missing values & rename columns

In [258]:
# Drop the duplicated rows
df.drop_duplicates(inplace=True)

In [259]:
# rename columns name 'locality' to 'postal_code'

df.rename(columns={'locality' :'postal_code'},inplace=True)

#df.rename(columns={"A": "a", "B": "c"})

In [260]:
# Move column: 'city_name' next to 'postal_code'

second_col = df.pop("city_name")

df.insert(1, "city_name", second_col)


In [261]:
# Drop columns with only 1 unique value

df.describe(include='object') 

df.drop(columns=['type_of_sale','furnished','surface_of_the_plot_of_land'],inplace=True)
df.shape

,city_name,type_of_property,subtype_of_property,price,type_of_sale,house_area,furnished,terrace_area,garden_area,surface_of_the_plot_of_land,number_of_facades,state_of_the_building,construction_year
count,50841,50841,50841,50841,50841,50841,50841,50841,50841,50841,50841,50841,50841
unique,1070,2,24,4220,1,693,1,185,1192,1,5,7,220
top,Antwerpen,house,house,249000,for sale,None,None,None,None,None,2,as new,None
freq,1028,29001,21809,674,50841,8430,50841,33745,42491,50841,17834,14358,20958


(50841, 20)

In [262]:
df.isna().sum()

postal_code                   0
city_name                     0
type_of_property              0
subtype_of_property           0
price                         0
number_of_rooms               2
house_area                    0
fully_equipped_kitchen        0
open_fire                     0
terrace                       0
terrace_area                  0
garden                        0
garden_area                   0
surface_of_the_land       21840
number_of_facades             0
swimming_pool                 0
state_of_the_building         0
construction_year             0
lattitude                     0
longitude                     0
dtype: int64

In [263]:
# fill missing value as "None" for columns: surface_of_the_land
df.surface_of_the_land = df.surface_of_the_land.fillna('None')

In [264]:
df.isna().sum()

postal_code               0
city_name                 0
type_of_property          0
subtype_of_property       0
price                     0
number_of_rooms           2
house_area                0
fully_equipped_kitchen    0
open_fire                 0
terrace                   0
terrace_area              0
garden                    0
garden_area               0
surface_of_the_land       0
number_of_facades         0
swimming_pool             0
state_of_the_building     0
construction_year         0
lattitude                 0
longitude                 0
dtype: int64

In [265]:
# drop 2 rows of missing value for columns: number_of_rooms)

df.dropna(inplace=True)

In [266]:
df.isna().sum()

postal_code               0
city_name                 0
type_of_property          0
subtype_of_property       0
price                     0
number_of_rooms           0
house_area                0
fully_equipped_kitchen    0
open_fire                 0
terrace                   0
terrace_area              0
garden                    0
garden_area               0
surface_of_the_land       0
number_of_facades         0
swimming_pool             0
state_of_the_building     0
construction_year         0
lattitude                 0
longitude                 0
dtype: int64

## Columns cleaning

In [267]:
# Function to clean unique and dtype of a given column.
def print_unique_dtype(column):
  """Print unique and dtype of a given column"""
  
  unique = df[column].value_counts().nunique()
  dtype = df[column].dtype

  print(f"{unique} - {dtype}")

### postal_code 

In [268]:
# Postal Code is used earlier to aggregate the city name and regions,
# so we know it's already clean.

print_unique_dtype('postal_code')

195 - int64


### city_name

In [269]:
# City name is an "object", it come from the aggregated city name,
# so we know it's already clean -> Str.

print_unique_dtype('city_name')

195 - object


### type_of_property

In [270]:
# Type of property is currently a string:
# either 'house' or 'apartment'

print_unique_dtype('type_of_property')
df['type_of_property'].value_counts()

2 - object


house        29000
apartment    21839
Name: type_of_property, dtype: int64

In [271]:
# Change house to 0, and apartment to 1

df['type_of_property'] = df['type_of_property'].replace({"house":0 ,"apartment":1})
df.head()

,postal_code,city_name,type_of_property,subtype_of_property,price,number_of_rooms,house_area,fully_equipped_kitchen,open_fire,terrace,terrace_area,garden,garden_area,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,construction_year,lattitude,longitude
0,1050,Ixelles,0,house,340000,6.0,203,1,0,1,None,0,None,95,2,0,to be done up,1901,4.381571,50.822285
1,1050,Ixelles,0,mixed use building,520000,4.0,200,0,0,0,None,0,None,69,2,0,to renovate,1940,4.381571,50.822285
2,1050,Ixelles,0,house,no price,6.0,None,1,0,0,None,0,None,0,None,0,None,1910,4.381571,50.822285
3,1050,Ixelles,0,house,599000,4.0,160,1,0,1,None,1,55,100,2,0,to be done up,1898,4.381571,50.822285
4,1050,Ixelles,0,house,599000,3.0,160,1,0,1,15,1,60,130,2,0,good,1953,4.381571,50.822285


### subtype_of_property

In [272]:
# Subtype of property is related to the property's type.
# We'll drop "castle" and "pavillon"

print_unique_dtype('subtype_of_property')
df.subtype_of_property.value_counts()

23 - object


house                   21809
apartment               16904
villa                    2796
duplex                   1371
ground floor             1170
apartment block          1038
mixed use building        983
penthouse                 886
flat studio               852
mansion                   459
exceptional property      447
town house                409
country cottage           317
service flat              284
bungalow                  280
loft                      216
farmhouse                 143
chalet                    138
other property             85
manor house                85
kot                        83
triplex                    73
castle                     10
pavilion                    1
Name: subtype_of_property, dtype: int64

In [273]:
# Drop castle & pavilion & apartment block.
df = df[(df['subtype_of_property'] != 'castle') & (df['subtype_of_property'] != 'pavilion') & (df['subtype_of_property'] != 'apartment block')]

print_unique_dtype('subtype_of_property')
df.shape

20 - object


(49790, 20)

In [274]:
# Drop the columns
df.drop(columns=['subtype_of_property'],inplace=True)

### price

In [275]:
# Remove the "no price" string from price column.
df['price'] = df[df.price != "no price"]['price']
df.dropna(inplace=True)

# Convert the price to int
df['price'] = df['price'].astype(int)


df.shape
df['price'].dtype

(49722, 19)

dtype('int64')

### number_of_rooms

In [276]:
# 204 and 165 are too many rooms to be real.
# We remove it.

df.number_of_rooms.value_counts()

3.0      16684
2.0      16077
4.0       7081
1.0       4938
5.0       2441
0.0        985
6.0        927
7.0        270
8.0        126
9.0         62
10.0        57
11.0        22
12.0        15
15.0         7
13.0         5
16.0         4
20.0         3
14.0         3
18.0         3
204.0        3
17.0         2
30.0         2
23.0         1
165.0        1
33.0         1
24.0         1
19.0         1
Name: number_of_rooms, dtype: int64

In [277]:
# Remove number_of_rooms >= 165
df = df[(df['number_of_rooms'] < 20) & (df['number_of_rooms'] > 0)]

# Convert to int
df['number_of_rooms'] = df['number_of_rooms'].astype(int)

df['number_of_rooms'].value_counts()
df.shape

3     16684
2     16077
4      7081
1      4938
5      2441
6       927
7       270
8       126
9        62
10       57
11       22
12       15
15        7
13        5
16        4
18        3
14        3
17        2
19        1
Name: number_of_rooms, dtype: int64

(48725, 19)

### house_area

In [278]:
# Lots of None values (8427 entries)
df.house_area.value_counts()


None    7921
90       904
120      897
100      886
150      814
        ... 
1013       1
1350       1
607        1
639        1
527        1
Name: house_area, Length: 661, dtype: int64

In [279]:
# Remove the 'None' (str) entries
df = df[(df['house_area'] != 'None')]

# Convert the type to int
df['house_area'] = df['house_area'].astype(int)

# print_unique_dtype('subtype_of_property')
df.shape
df['house_area'].dtype

(40804, 19)

dtype('int64')

In [280]:
# Delete the 1 and 31700 entries 
df = df[(df['house_area'] != 1) & (df['house_area'] != 31700)]

df.house_area.max()


3560

### fully_equipped_kitchen

In [281]:
# fully_equiped_kitchen is clean

df.fully_equipped_kitchen.value_counts() 
df.fully_equipped_kitchen.dtype

1    28504
0    12295
Name: fully_equipped_kitchen, dtype: int64

dtype('int64')

### open_fire

In [282]:
# open_fire is clean

df.open_fire.value_counts() 
df.open_fire.dtype

0    38622
1     2177
Name: open_fire, dtype: int64

dtype('int64')

### terrace

In [283]:
# terrace is clean

df.terrace.value_counts()
df.terrace.dtype

1    25236
0    15563
Name: terrace, dtype: int64

dtype('int64')

### terrace_area

In [284]:
# 66.4% (25 000) entries are None.
# We can't remove them.

# Remove the terrace_area field ?
# Replace the terrace_area by 'small'/'medium'/'large' terrace, and assign all None to 'small' ?
# Replace by 0 ?

#Finally the columns will be dropped

df.drop(columns=['terrace_area'],inplace=True)


In [285]:
# Replace: None = 0
# df['terrace_area'] = df['terrace_area'].replace({"None":0})

# df.terrace_area.value_counts() 

### garden

In [286]:
# garden is clean

df.garden.value_counts()
df.garden.dtype

0    27706
1    13093
Name: garden, dtype: int64

dtype('int64')

### garden_area

In [287]:
# 33 000 'None' entries
# df.garden_area.value_counts() 

df.drop(columns=['garden_area'],inplace=True)


In [288]:
# Replace 'None' by 0
# df['garden_area'] = df['garden_area'].replace({"None":0})

# df.terrace_area.value_counts() 

### surface_of_the_land

In [289]:
# Surface of the land has 20 000 None entries
df.surface_of_the_land.value_counts()

None       19236
0.0         1797
150.0        170
200.0        160
1000.0       146
           ...  
74400.0        1
2229.0         1
2228.0         1
1018.0         1
2673.0         1
Name: surface_of_the_land, Length: 2955, dtype: int64

In [290]:
# Replace 'None' by 0
df['surface_of_the_land'] = df['surface_of_the_land'].replace({"None":0}).astype(int)

df.surface_of_the_land.value_counts()

0        21033
150        170
200        160
1000       146
300        146
         ...  
15222        1
2992         1
33793        1
1041         1
2737         1
Name: surface_of_the_land, Length: 2954, dtype: int64

### number_of_facades

In [291]:
# Number of facades has a lot of None value.
# Also, some house have only 1 facade.

df.number_of_facades.value_counts()

2       14531
None    10360
4        8104
3        7400
1         404
Name: number_of_facades, dtype: int64

In [292]:
# Replace 'None' by 0
df['number_of_facades'] = df['number_of_facades'].replace({"None": 0}).astype(int)

# Drop the 1-facade houses.
df = df[(df['number_of_facades'] != 1)]

df.number_of_facades.value_counts()

2    14531
0    10360
4     8104
3     7400
Name: number_of_facades, dtype: int64

### swimming_pool

In [293]:
# Swimming pool data is clean
df.swimming_pool.value_counts()

0    39699
1      696
Name: swimming_pool, dtype: int64

### state_of_the_building

In [294]:
# State of the building data has 'None'
df.state_of_the_building.value_counts()

as new            12096
good              10985
None               9796
to be done up      2789
to renovate        2441
just renovated     2147
to restore          141
Name: state_of_the_building, dtype: int64

In [295]:
# Replace 'None' by 'unknown'
df['state_of_the_building'] = df['state_of_the_building'].replace({"None": "unknown"})
df.state_of_the_building.value_counts()

as new            12096
good              10985
unknown            9796
to be done up      2789
to renovate        2441
just renovated     2147
to restore          141
Name: state_of_the_building, dtype: int64

### construction_year

In [296]:
df.construction_year.value_counts(normalize=True)

None    0.378710
2020    0.080456
2019    0.047852
2018    0.022206
1930    0.013021
          ...   
1781    0.000025
1855    0.000025
1876    0.000025
1854    0.000025
1873    0.000025
Name: construction_year, Length: 217, dtype: float64

In [297]:
# 41% of data construction year unknown.
# drop construction year column

df.drop(columns=['construction_year'],inplace=True)

## Provinces and regions aggregation

In [298]:
# This cell create a dictionnary of all postal codes, and their associated provinces and regions.

provinces = {
    'Bruxelles-Capitale': {
        'range': [(1000, 1299)],
        'region': 'Bruxelles'
    },
    'Brabant wallon': {
        'range': [(1300, 1499)],
        'region': 'Wallonie'
    },
    'Brabant flamand': {
        'range': [(1500, 1999), (3000, 3499)],
        'region': 'Flandre'
    },
    'Anvers': {
        'range': [(2000, 2999)],
        'region': 'Flandre'
    },
    'Limbourg': {
        'range': [(3500, 3999)],
        'region': 'Flandre'
    },
    'Liège': {
        'range': [(4000, 4999)],
        'region': 'Wallonie'
    },
    'Namur': {
        'range': [(5000, 5999)],
        'region': 'Wallonie'
    },
    'Hainaut': {
        'range': [(6000, 6599), (7000, 7999)],
        'region': 'Wallonie'
    },
    'Luxembourg': {
        'range': [(6600, 6999)],
        'region': 'Wallonie'
    },
    'Flandre-Occidentale': {
        'range': [(8000, 8999)],
        'region': 'Flandre'
    },
    'Flandre-Orientale': {
        'range': [(9000, 9999)],
        'region': 'Flandre'
    }
}

def convert_by_postal_code():
  """
  Convert the 'provinces' dictionnary to a dictionnary of postal codes as key,
  and its region and province as values.
  """
  postal_dict = {}

  # Loop through each entry in the "provinces" dictionnary
  for province, content in provinces.items():

    # Loop though each postal code
    for postal_code_tuple in content['range']:
      for code in range(postal_code_tuple[0], postal_code_tuple[1] + 1):

        # Append the province and region for each postal code.
        postal_dict[code] = {'province': province, 'region': content['region']}

  return postal_dict

postal_dict = convert_by_postal_code()
print(postal_dict)


{1000: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1001: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1002: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1003: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1004: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1005: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1006: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1007: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1008: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1009: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1010: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1011: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1012: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1013: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1014: {'province': 'Bruxelles-Capitale', 'region': 'Bruxelles'}, 1015: {'province': 'Brux

In [299]:
def add_province(postal_code):
  """Return the province associated with a given locality."""
  return postal_dict[postal_code]['province']

def add_region(postal_code):
  return postal_dict[postal_code]['region']

provinces_regions = df['postal_code'].apply(lambda postal_code: pd.Series({
    'province': add_province(postal_code),
    'region': add_region(postal_code)
    }))

df = pd.concat([df, provinces_regions], axis=1)
df.head(10)

,postal_code,city_name,type_of_property,price,number_of_rooms,house_area,fully_equipped_kitchen,open_fire,terrace,garden,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,lattitude,longitude,province,region
0,1050,Ixelles,0,340000,6,203,1,0,1,0,95,2,0,to be done up,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
1,1050,Ixelles,0,520000,4,200,0,0,0,0,69,2,0,to renovate,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
3,1050,Ixelles,0,599000,4,160,1,0,1,1,100,2,0,to be done up,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
4,1050,Ixelles,0,599000,3,160,1,0,1,1,130,2,0,good,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
5,1050,Ixelles,0,575000,3,171,0,0,0,0,46,2,0,just renovated,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
6,1050,Ixelles,0,590000,4,225,0,0,1,0,0,2,0,to renovate,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
7,1050,Ixelles,0,575000,4,209,1,0,0,0,0,2,0,unknown,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
8,1050,Ixelles,0,595000,1,195,1,1,1,1,617,4,0,as new,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
9,1050,Ixelles,0,595777,4,250,0,0,0,0,70,2,0,unknown,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
11,1050,Ixelles,0,650000,6,250,1,0,0,0,60,2,0,good,4.381571,50.822285,Bruxelles-Capitale,Bruxelles


## Export

In [300]:
df.dropna(inplace=True)
df.shape

df.to_csv('belgium_real_estate.csv')

(40395, 18)

In [301]:
df

,postal_code,city_name,type_of_property,price,number_of_rooms,house_area,fully_equipped_kitchen,open_fire,terrace,garden,surface_of_the_land,number_of_facades,swimming_pool,state_of_the_building,lattitude,longitude,province,region
0,1050,Ixelles,0,340000,6,203,1,0,1,0,95,2,0,to be done up,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
1,1050,Ixelles,0,520000,4,200,0,0,0,0,69,2,0,to renovate,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
3,1050,Ixelles,0,599000,4,160,1,0,1,1,100,2,0,to be done up,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
4,1050,Ixelles,0,599000,3,160,1,0,1,1,130,2,0,good,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
5,1050,Ixelles,0,575000,3,171,0,0,0,0,46,2,0,just renovated,4.381571,50.822285,Bruxelles-Capitale,Bruxelles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52070,1472,Vieux-Genappe,0,475000,5,216,1,1,0,0,1550,4,1,as new,4.401503,50.629025,Brabant wallon,Wallonie
52071,1472,Vieux-Genappe,0,475000,5,215,1,0,1,0,1550,0,1,good,4.401503,50.629025,Brabant wallon,Wallonie
52072,1461,Haut-Ittre,0,499000,5,275,1,0,1,1,1561,4,0,unknown,4.296472,50.648804,Brabant wallon,Wallonie
52073,1761,Borchtlombeek,0,495000,4,235,1,0,0,1,488,4,0,unknown,4.136915,50.848178,Brabant flamand,Flandre
